# TP 06 V3 : Fine-tuning GPT-2 - Version Pédagogique

**Objectif** : Fine-tuner GPT-2 français pour générer des descriptions de Pokémon

**Structure V3** :
1. **Exploration** : Découvrir le tokenizer, le dataset, les techniques
2. **Configuration** : Tous les paramètres au même endroit (juste avant l'entraînement)
3. **Entraînement** : Lancer le fine-tuning

**Durée** : 2h

---

# PARTIE 1 : EXPLORATION

Dans cette partie, on explore les données et les techniques sans encore lancer l'entraînement.

## 1.0 Installation et imports

In [ ]:
# Installation des dépendances (Colab)
!pip install -q transformers datasets accelerate

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, TrainerCallback
from datasets import load_dataset, concatenate_datasets
import matplotlib.pyplot as plt
import random
import warnings
warnings.filterwarnings('ignore')

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# Reproductibilité
torch.manual_seed(42)
random.seed(42)

---

## 1.1 Explorer le tokenizer GPT-2 français

Voyons comment GPT-2 tokenise du texte, notamment les noms de Pokémon.

In [ ]:
# Charger le tokenizer pour exploration
print("Chargement du tokenizer GPT-2 français...")
tokenizer_demo = AutoTokenizer.from_pretrained("asi/gpt-fr-cased-base")
print(f"Vocabulaire : {len(tokenizer_demo):,} tokens")

In [ ]:
# Comment le tokenizer découpe-t-il les noms de Pokémon ?
test_words = ["Pikachu", "Dracaufeu", "Salamèche", "Bulbizarre", "Mewtwo", "Évolution"]

print("═" * 50)
print("Comment GPT-2 tokenise les noms de Pokémon ?")
print("═" * 50)

for word in test_words:
    tokens = tokenizer_demo.tokenize(word)
    ids = tokenizer_demo.encode(word, add_special_tokens=False)
    status = "✅ Token unique" if len(tokens) == 1 else f"❌ {len(tokens)} sous-tokens"
    print(f"'{word}' → {tokens} (ids: {ids}) - {status}")

In [ ]:
# Vérifier si un mot existe dans le vocabulaire
def check_vocab(word, tokenizer):
    """Vérifie si un mot est un token unique dans le vocabulaire."""
    vocab = tokenizer.get_vocab()
    # Tester avec et sans le préfixe Ġ (espace)
    in_vocab = word in vocab or f"Ġ{word}" in vocab
    tokens = tokenizer.tokenize(word)
    return {
        "word": word,
        "in_vocab": in_vocab,
        "tokens": tokens,
        "is_single": len(tokens) == 1
    }

# Test
result = check_vocab("Pikachu", tokenizer_demo)
print(f"Pikachu dans le vocabulaire : {result}")

**Observation** : La plupart des noms de Pokémon sont découpés en plusieurs sous-tokens par BPE.

C'est pourquoi on peut vouloir les ajouter au vocabulaire (voir section Configuration).

---

## 1.2 Explorer le dataset Pokémon

In [ ]:
# Charger les datasets
print("Chargement des datasets...")

# Dataset principal (articles Pokepedia)
dataset_pokepedia = load_dataset("chris-lmd/pokepedia-fr")
print(f"\n📚 Pokepedia : {len(dataset_pokepedia['train']):,} articles")

# Liste des noms de Pokémon
dataset_names = load_dataset("chris-lmd/pokemon-names-fr")
POKEMON_NAMES = [item["name"] for item in dataset_names["train"]]
print(f"📋 Noms de Pokémon : {len(POKEMON_NAMES)}")
print(f"   Exemples : {POKEMON_NAMES[:10]}")

In [ ]:
# Aperçu d'un article
sample = dataset_pokepedia['train'][0]
print(f"Titre: {sample.get('title', 'N/A')}")
print(f"\nContenu (500 premiers caractères):")
print(sample['content'][:500] + "...")

In [ ]:
# Analyse : combien d'articles parlent vraiment de Pokémon ?
pokemon_names_set = set(name.lower() for name in POKEMON_NAMES)

def is_pokemon_article(example):
    """Article dont le titre est un nom de Pokémon."""
    title = example.get('title', '').lower()
    return title in pokemon_names_set

pokemon_articles = dataset_pokepedia['train'].filter(is_pokemon_article)
other_articles = dataset_pokepedia['train'].filter(lambda x: not is_pokemon_article(x))

print("═" * 50)
print("Analyse du dataset")
print("═" * 50)
print(f"Total articles : {len(dataset_pokepedia['train']):,}")
print(f"Articles Pokémon (titre = nom) : {len(pokemon_articles):,}")
print(f"Autres articles : {len(other_articles):,}")

In [ ]:
# Exemples d'articles Pokémon vs autres
print("\n📍 Exemples d'articles POKÉMON :")
for i in range(min(5, len(pokemon_articles))):
    print(f"  - {pokemon_articles[i]['title']}")

print("\n📍 Exemples d'AUTRES articles :")
for i in range(min(5, len(other_articles))):
    print(f"  - {other_articles[i]['title']}")

---

## 1.3 Comprendre les techniques d'optimisation

### Technique 1 : Ajouter des tokens au vocabulaire (Smart Initialization)

**Problème** : "Pikachu" est découpé en sous-tokens → le modèle doit apprendre à reconstituer le concept.

**Solution** : Ajouter "Pikachu" comme token unique, initialisé avec l'embedding de "Pokémon".

In [ ]:
# Combien de noms de Pokémon ne sont PAS des tokens uniques ?
tokens_to_add = []
tokens_already_exist = []

for name in POKEMON_NAMES:
    tokens = tokenizer_demo.encode(name, add_special_tokens=False)
    if len(tokens) > 1:
        tokens_to_add.append(name)
    else:
        tokens_already_exist.append(name)

print(f"Noms déjà dans le vocabulaire : {len(tokens_already_exist)}")
print(f"Noms à ajouter (multi-tokens) : {len(tokens_to_add)}")
print(f"\nExemples de noms à ajouter : {tokens_to_add[:10]}")

### Technique 2 : Figer les couches basses (Partial Freezing)

**Principe** : Les couches basses capturent la grammaire générale (déjà bien apprise). On ne fine-tune que les couches hautes (sémantique).

**Avantages** : Plus rapide, moins d'overfitting.

In [ ]:
# Visualiser l'architecture GPT-2
print("═" * 50)
print("Architecture GPT-2")
print("═" * 50)
print()
print("┌─────────────────────────────┐")
print("│   Token Embeddings (wte)    │ ← Représentation des mots")
print("├─────────────────────────────┤")
print("│ Position Embeddings (wpe)   │ ← Position dans la séquence")
print("├─────────────────────────────┤")
print("│   Transformer Block 0      │")
print("│   Transformer Block 1      │ ← Couches BASSES")
print("│          ...               │   (grammaire, syntaxe)")
print("│   Transformer Block N/2    │   → À FIGER")
print("├─────────────────────────────┤")
print("│   Transformer Block N/2+1  │")
print("│          ...               │ ← Couches HAUTES")
print("│   Transformer Block N-1    │   (sémantique, style)")
print("├─────────────────────────────┤   → À ENTRAÎNER")
print("│      Layer Norm (ln_f)      │")
print("├─────────────────────────────┤")
print("│        LM Head             │ ← Prédiction du prochain token")
print("└─────────────────────────────┘")
print()
print("Modèle small : 12 couches")
print("Modèle base  : 24 couches")

---

# PARTIE 2 : CONFIGURATION ET ENTRAÎNEMENT

Maintenant que nous avons exploré les données et les techniques, configurons et lançons l'entraînement.

---

## 2.1 ⚙️ CONFIGURATION CENTRALE

**Tous les paramètres sont ici.** Modifiez cette cellule puis exécutez les suivantes.

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║                    CONFIGURATION CENTRALE                        ║
# ║         Modifiez ces paramètres puis exécutez la suite          ║
# ╚══════════════════════════════════════════════════════════════════╝

# ─────────────────────────────────────────────────────────────────────
# MODÈLE
# ─────────────────────────────────────────────────────────────────────
MODEL_SIZE = "base"      # "small" (12 couches, rapide) ou "base" (24 couches, meilleur)

# ─────────────────────────────────────────────────────────────────────
# TECHNIQUES D'OPTIMISATION
# ─────────────────────────────────────────────────────────────────────
ADD_POKEMON_TOKENS = True     # Ajouter les noms de Pokémon au vocabulaire
FREEZE_LOWER_LAYERS = True    # Figer les couches basses
FREEZE_RATIO = 0.5            # Proportion de couches à figer (0.5 = 50%)

# ─────────────────────────────────────────────────────────────────────
# DATASET
# ─────────────────────────────────────────────────────────────────────
MAX_SAMPLES = 5000       # Nombre d'articles pour l'entraînement
MAX_LENGTH = 256         # Longueur max des séquences (tokens)
PRIORITIZE_POKEMON = True     # Prioriser les vrais articles Pokémon

# ─────────────────────────────────────────────────────────────────────
# ENTRAÎNEMENT
# ─────────────────────────────────────────────────────────────────────
NUM_EPOCHS = 3           # Nombre d'epochs
LEARNING_RATE = 5e-5     # Taux d'apprentissage
BATCH_SIZE = None        # None = automatique selon le modèle

# ─────────────────────────────────────────────────────────────────────
# Affichage de la configuration
# ─────────────────────────────────────────────────────────────────────
print("╔" + "═" * 58 + "╗")
print("║" + " CONFIGURATION ACTIVE ".center(58) + "║")
print("╠" + "═" * 58 + "╣")
print(f"║  Modèle          : {MODEL_SIZE:<37} ║")
print(f"║  Tokens Pokémon  : {str(ADD_POKEMON_TOKENS):<37} ║")
print(f"║  Freeze layers   : {str(FREEZE_LOWER_LAYERS):<37} ║")
print(f"║  Freeze ratio    : {FREEZE_RATIO:<37} ║")
print(f"║  Max samples     : {MAX_SAMPLES:<37} ║")
print(f"║  Max length      : {MAX_LENGTH:<37} ║")
print(f"║  Epochs          : {NUM_EPOCHS:<37} ║")
print(f"║  Learning rate   : {LEARNING_RATE:<37} ║")
print("╚" + "═" * 58 + "╝")

---

## 2.2 Chargement du modèle et tokenizer

In [ ]:
# Charger le modèle selon la configuration
model_name = f"asi/gpt-fr-cased-{MODEL_SIZE}"

print(f"Chargement du modèle : {model_name}")
print("Chargement du tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

print("Chargement du modèle...")
model = AutoModelForCausalLM.from_pretrained(model_name)

# Ajouter un token de padding
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

print(f"\n✅ Modèle chargé !")
print(f"   Paramètres : {sum(p.numel() for p in model.parameters()):,}")
print(f"   Vocabulaire : {len(tokenizer):,} tokens")
print(f"   Couches : {model.config.n_layer}")

---

## 2.3 Application des techniques d'optimisation

In [ ]:
# ═══════════════════════════════════════════════════════════════
# TECHNIQUE 1 : Ajouter les tokens Pokémon
# ═══════════════════════════════════════════════════════════════

def add_pokemon_tokens_to_vocab(tokenizer, model, pokemon_names):
    """Ajoute les noms de Pokémon au vocabulaire avec smart initialization."""
    
    # Trouver le token de référence
    reference_tokens = ["Pokémon", "Pokemon", "animal", "créature"]
    reference_id = None
    reference_word = None
    
    for ref in reference_tokens:
        tokens = tokenizer.encode(ref, add_special_tokens=False)
        if len(tokens) == 1:
            reference_id = tokens[0]
            reference_word = ref
            break
    
    if reference_id is None:
        reference_id = tokenizer.encode("Pokémon", add_special_tokens=False)[0]
        reference_word = "Pokémon (premier sous-token)"
    
    print(f"   Token de référence : '{reference_word}' (id={reference_id})")
    
    # Filtrer les noms qui ne sont pas déjà des tokens uniques
    new_tokens = [name for name in pokemon_names 
                  if len(tokenizer.encode(name, add_special_tokens=False)) > 1]
    
    print(f"   Tokens à ajouter : {len(new_tokens)}")
    
    if len(new_tokens) == 0:
        return 0
    
    # Sauvegarder l'embedding de référence
    with torch.no_grad():
        reference_embedding = model.transformer.wte.weight[reference_id].clone()
    
    # Ajouter les tokens
    num_added = tokenizer.add_tokens(new_tokens)
    old_size = model.transformer.wte.weight.shape[0]
    model.resize_token_embeddings(len(tokenizer))
    
    # Initialiser avec l'embedding de référence + bruit
    with torch.no_grad():
        for i in range(num_added):
            new_token_id = old_size + i
            noise = torch.randn_like(reference_embedding) * 0.01
            model.transformer.wte.weight[new_token_id] = reference_embedding + noise
    
    return num_added


if ADD_POKEMON_TOKENS:
    print("═" * 50)
    print("TECHNIQUE 1 : Ajout des tokens Pokémon")
    print("═" * 50)
    
    # Test AVANT
    tokens_before = tokenizer.tokenize("Pikachu")
    print(f"\n   AVANT : 'Pikachu' → {tokens_before}")
    
    # Ajout des tokens
    num_added = add_pokemon_tokens_to_vocab(tokenizer, model, POKEMON_NAMES)
    
    # Test APRÈS
    tokens_after = tokenizer.tokenize("Pikachu")
    print(f"   APRÈS : 'Pikachu' → {tokens_after}")
    print(f"\n   ✅ {num_added} tokens ajoutés au vocabulaire")
else:
    print("❌ Option ADD_POKEMON_TOKENS désactivée")

In [ ]:
# ═══════════════════════════════════════════════════════════════
# TECHNIQUE 2 : Figer les couches basses
# ═══════════════════════════════════════════════════════════════

def freeze_lower_layers(model, freeze_ratio):
    """Fige les embeddings et les N premières couches."""
    total_layers = model.config.n_layer
    num_to_freeze = int(total_layers * freeze_ratio)
    
    # Figer les embeddings
    for param in model.transformer.wte.parameters():
        param.requires_grad = False
    for param in model.transformer.wpe.parameters():
        param.requires_grad = False
    
    # Figer les N premières couches
    for i in range(num_to_freeze):
        for param in model.transformer.h[i].parameters():
            param.requires_grad = False
    
    # Stats
    total_params = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    frozen = total_params - trainable
    
    return num_to_freeze, trainable, frozen, total_params


if FREEZE_LOWER_LAYERS:
    print("\n" + "═" * 50)
    print("TECHNIQUE 2 : Freezing des couches basses")
    print("═" * 50)
    
    num_frozen, trainable, frozen, total = freeze_lower_layers(model, FREEZE_RATIO)
    
    print(f"\n   Couches figées : {num_frozen} / {model.config.n_layer}")
    print(f"   Paramètres figés : {frozen:,} ({100*frozen/total:.1f}%)")
    print(f"   Paramètres entraînables : {trainable:,} ({100*trainable/total:.1f}%)")
else:
    print("\n❌ Option FREEZE_LOWER_LAYERS désactivée")
    print("   Tous les paramètres seront entraînés (full fine-tuning)")

In [ ]:
# Déplacer le modèle sur GPU
model = model.to(device)
print(f"\n✅ Modèle déplacé sur : {device}")

---

## 2.4 Test du modèle AVANT fine-tuning

In [ ]:
def generate_text(prompt, max_length=100, temperature=0.8):
    """Génère du texte à partir d'un prompt."""
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            temperature=temperature,
            top_k=50,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test
prompt = "Pikachu est un Pokémon de type"
print(f"Prompt: {prompt}")
print(f"\nGénération AVANT fine-tuning:")
print(generate_text(prompt))

---

## 2.5 Préparation du dataset

In [ ]:
# Constitution du dataset d'entraînement
if PRIORITIZE_POKEMON:
    # Prioriser les vrais articles Pokémon
    num_pokemon = len(pokemon_articles)
    
    if num_pokemon >= MAX_SAMPLES:
        train_dataset = pokemon_articles.shuffle(seed=42).select(range(MAX_SAMPLES))
        print(f"✅ Dataset : {MAX_SAMPLES} vrais Pokémon")
    else:
        # Compléter avec d'autres articles
        remaining = MAX_SAMPLES - num_pokemon
        other_sample = other_articles.shuffle(seed=42).select(range(min(remaining, len(other_articles))))
        train_dataset = concatenate_datasets([pokemon_articles, other_sample]).shuffle(seed=42)
        print(f"✅ Dataset : {len(train_dataset)} articles")
        print(f"   - Vrais Pokémon : {num_pokemon}")
        print(f"   - Autres : {len(other_sample)}")
else:
    # Échantillon aléatoire
    train_dataset = dataset_pokepedia['train'].shuffle(seed=42).select(range(MAX_SAMPLES))
    print(f"✅ Dataset : {MAX_SAMPLES} articles (aléatoire)")

In [ ]:
# Tokenization
def tokenize_function(examples):
    return tokenizer(
        examples['content'],
        truncation=True,
        max_length=MAX_LENGTH,
        padding='max_length'
    )

print("Tokenization en cours...")
tokenized_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

# Ajouter les labels
def add_labels(examples):
    examples['labels'] = examples['input_ids'].copy()
    return examples

tokenized_dataset = tokenized_dataset.map(add_labels, batched=True)
print(f"✅ Tokenization terminée !")

---

## 2.6 Lancement de l'entraînement

In [ ]:
# Callback pour enregistrer la loss à chaque step
class LossHistoryCallback(TrainerCallback):
    def __init__(self):
        self.loss_history = []
        self.steps = []
    
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None and "loss" in logs:
            self.loss_history.append(logs["loss"])
            self.steps.append(state.global_step)

# Créer le callback
loss_callback = LossHistoryCallback()

# Batch size adapté au modèle
if BATCH_SIZE is None:
    batch_size = 2 if MODEL_SIZE == "base" else 4
    grad_accum = 8 if MODEL_SIZE == "base" else 4
else:
    batch_size = BATCH_SIZE
    grad_accum = 4

training_args = TrainingArguments(
    output_dir="./gpt2-pokemon-v3",
    overwrite_output_dir=True,
    
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=grad_accum,
    
    learning_rate=LEARNING_RATE,
    warmup_steps=100,
    weight_decay=0.01,
    
    logging_steps=10,  # Log plus fréquent pour la courbe
    save_steps=500,
    save_total_limit=2,
    
    fp16=torch.cuda.is_available(),
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    callbacks=[loss_callback],  # Ajouter le callback
)

print("✅ Trainer créé avec monitoring de la loss !")

In [ ]:
# Résumé final avant entraînement
print("╔" + "═" * 58 + "╗")
print("║" + " RÉSUMÉ - PRÊT POUR L'ENTRAÎNEMENT ".center(58) + "║")
print("╠" + "═" * 58 + "╣")
print(f"║  🤖 Modèle : {model_name:<43} ║")
print(f"║  📊 Dataset : {len(train_dataset):,} articles{' ':<33} ║")
print(f"║  📝 Max length : {MAX_LENGTH} tokens{' ':<32} ║")
print(f"║  🔄 Epochs : {NUM_EPOCHS}{' ':<44} ║")
print(f"║  📦 Batch : {batch_size} x {grad_accum} = {batch_size * grad_accum} effectif{' ':<25} ║")
print("╠" + "═" * 58 + "╣")
print(f"║  ✨ Tokens Pokémon : {str(ADD_POKEMON_TOKENS):<36} ║")
frozen_layers = int(model.config.n_layer * FREEZE_RATIO) if FREEZE_LOWER_LAYERS else 0
print(f"║  🧊 Couches figées : {frozen_layers}/{model.config.n_layer}{' ':<35} ║")
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"║  🧠 Params entraînables : {trainable:,} ({100*trainable/total:.0f}%){' ':<14} ║")
print("╚" + "═" * 58 + "╝")

In [ ]:
# 🚀 LANCEMENT DE L'ENTRAÎNEMENT
print("🚀 Fine-tuning en cours...")
print()

trainer.train()

print("\n✅ Fine-tuning terminé !")

### 📊 Analyse de la loss

**Comment interpréter la courbe :**
- **Descente régulière** → Bon apprentissage
- **Plateau** → Le modèle a convergé, plus d'epochs inutiles
- **Remontée** → Overfitting ! Arrêter l'entraînement
- **Oscillations fortes** → Learning rate trop élevé

In [ ]:
# 📊 Courbe de la loss
plt.figure(figsize=(12, 5))

# Subplot 1 : Loss complète
plt.subplot(1, 2, 1)
plt.plot(loss_callback.steps, loss_callback.loss_history, 'b-', alpha=0.7, linewidth=1)
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.grid(True, alpha=0.3)

# Subplot 2 : Loss lissée (moyenne mobile)
plt.subplot(1, 2, 2)
window = min(10, len(loss_callback.loss_history) // 5) if len(loss_callback.loss_history) > 10 else 1
if window > 1:
    smoothed = []
    for i in range(len(loss_callback.loss_history)):
        start = max(0, i - window)
        smoothed.append(sum(loss_callback.loss_history[start:i+1]) / (i - start + 1))
    plt.plot(loss_callback.steps, smoothed, 'r-', linewidth=2, label=f'Moyenne mobile (window={window})')
    plt.plot(loss_callback.steps, loss_callback.loss_history, 'b-', alpha=0.3, linewidth=1, label='Loss brute')
    plt.legend()
else:
    plt.plot(loss_callback.steps, loss_callback.loss_history, 'b-', linewidth=2)
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Training Loss (lissée)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Stats
print(f"\n📈 Statistiques :")
print(f"   Loss initiale : {loss_callback.loss_history[0]:.4f}")
print(f"   Loss finale   : {loss_callback.loss_history[-1]:.4f}")
print(f"   Réduction     : {(1 - loss_callback.loss_history[-1]/loss_callback.loss_history[0])*100:.1f}%")
print(f"   Loss min      : {min(loss_callback.loss_history):.4f} (step {loss_callback.steps[loss_callback.loss_history.index(min(loss_callback.loss_history))]})")

In [ ]:
# Sauvegarder le modèle
trainer.save_model("./gpt2-pokemon-v3-final")
tokenizer.save_pretrained("./gpt2-pokemon-v3-final")
print("✅ Modèle sauvegardé !")

---

# PARTIE 3 : GÉNÉRATION ET TESTS

In [ ]:
def generate_description(prompt, max_length=150, temperature=0.8):
    """Génère une description de Pokémon."""
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            temperature=temperature,
            top_k=50,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# Test : Pikachu
prompt = "Pikachu est un Pokémon de type"
print(f"Prompt: {prompt}")
print(f"\nGénération APRÈS fine-tuning:")
print(generate_description(prompt))

In [ ]:
# Test : Pokémon inventé
prompt = "Flamador est un Pokémon de type Feu. Il"
print(f"Prompt: {prompt}")
print(f"\nGénération:")
print(generate_description(prompt))

In [ ]:
# Test : Évolution
prompt = "Dracaufeu évolue à partir de"
print(f"Prompt: {prompt}")
print(f"\nGénération:")
print(generate_description(prompt))

---

## Récapitulatif

### Configurations suggérées

| Scénario | MODEL_SIZE | ADD_TOKENS | FREEZE | Résultat |
|----------|-----------|------------|--------|----------|
| Test rapide | small | False | False | ~10 min |
| Équilibré | base | True | True | ~20 min, bon résultat |
| Qualité max | base | True | False | ~40 min, meilleur |

### Techniques apprises

1. **Smart Token Initialization** : Ajouter des tokens spécialisés
2. **Partial Freezing** : Figer les couches basses pour un entraînement efficace
3. **Dataset curation** : Prioriser les données pertinentes